# 프린터 큐

In [2]:
from collections import deque
t = int(input())
for _ in range(t):
    n, m = map(int, input().split())
    queue = deque(list(map(int, input().split()))) # 중요도

    cnt = 0

    idx_que = deque(list(range(n))) # 문서번호

    while queue:
        if queue[0] == max(queue):
            cnt += 1
            queue.popleft()
            if idx_que.popleft() == m:
                print(cnt)
        else:
            queue.append(queue.popleft())
            idx_que.append(idx_que.popleft())

3
1 0
5
1
4 2
1 2 3 4
2
6 0
1 1 9 1 1 1
5


# 톱니바퀴

클린코드
- https://inspirit941.tistory.com/entry/Python-%EB%B0%B1%EC%A4%80-14891-%ED%86%B1%EB%8B%88%EB%B0%94%ED%80%B4

- 회전하는 톱니바퀴의 왼쪽 톱니바퀴를 확인할 때
    - 회전톱니바퀴[-2] == 왼쪽톱니바퀴[2]

- 회전하는 톱니바퀴의 오른쪽 톱니바퀴를 확인할 때
    - 회전톱니바퀴[2] == 오른쪽톱니바퀴[-2]

In [82]:
from collections import deque

# 톱니바퀴의 상태
a = deque(input())
b = deque(input())
c = deque(input())
d = deque(input())

# 회전 횟수
k = int(input())
for _ in range(k):
    num, direction = map(int, input().split())

    # 전체 톱니바퀴
    gear = deque()
    gear.append(a)
    gear.append(b)
    gear.append(c)
    gear.append(d)

    # 시작 톱니바퀴
    start = num-1

    total = {x:0 for x in range(4)}
    total[start] = direction

    if num == 1:
        # 1-1) 오른쪽톱니바퀴 확인
        if gear[start][2] != gear[start+1][-2]:
            total[start+1] = -direction
        #1-3) 세번째, 네번째 톱니바퀴 확인
        if total[start+1] != 0:
            if gear[start+1][2] != gear[start+2][-2]:
                total[start+2] = -total[start+1]
            if total[start+2] != 0:
                if gear[start+2][2] != gear[start+3][-2]:
                    total[start+3] = -total[start+2]

    elif num == 2:
        # 1-1) 왼쪽톱니바퀴 확인(b,c,d)
        if gear[start][-2] != gear[start-1][2]: # 서로 다른 극
            total[start-1] = -direction
        # 1-2) 오른쪽톱니바퀴 확인(a,b,c)
        if gear[start][2] != gear[start+1][-2]:
            total[start+1] = -direction
        #1-3) 나머지 톱니바퀴 확인
        if gear[start+1][2] != 0:
            if gear[start+1][2] != gear[start+2][-2]:
                total[start+2] = -total[start+1]

    # 1. 톱니바퀴 확인
    elif num == 3:
        # 1-1) 왼쪽톱니바퀴 확인(b,c,d)
        if gear[start][-2] != gear[start-1][2]: # 서로 다른 극
            total[start-1] = -direction
        # 1-2) 오른쪽톱니바퀴 확인(a,b,c)
        if gear[start][2] != gear[start+1][-2]:
            total[start+1] = -direction
        #1-3) 나머지 톱니바퀴 확인
        if gear[start-1][-2] != 0:
            if gear[start-1][-2] != gear[start-2][2]:
                total[start-2] = -total[start-1]

    elif num == 4:
        # 1-1) 왼톱니바퀴 확인
        if gear[start][-2] != gear[start-1][2]:
            total[start-1] = -direction
        #1-3) 두번째, 첫번째 톱니바퀴 확인
        if total[start-1] != 0:
            if gear[start-1][-2] != gear[start-2][2]:
                total[start-2] = -total[start-1]
            if total[start-2] != 0:
                if gear[start-2][-2] != gear[start-3][2]:
                    total[start-3] = -total[start-2]

    # 2. 회전
    for num, i in total.items():
        gear[num].rotate(i)

ans = int(gear[0][0]) + (2 * int(gear[1][0])) + (4 * int(gear[2][0])) + (8 * int(gear[3][0]))
print(ans)

10101111
01111101
11001110
00000010
2
3 -1
1 1
7


# 인구 이동

인구 이동은 더 이상 아래 방법에 의해 인구 이동이 없을 때까지 지속
- 국경선을 공유하는 두 나라의 인구 차이가 L명 이상, R명 이하라면, 두 나라가 공유하는 국경선을 오늘 하루 동안 연다.
- 위의 조건에 의해 열어야하는 국경선이 모두 열렸다면, 인구 이동을 시작한다.
- 국경선이 열려있어 인접한 칸만을 이용해 이동할 수 있으면, 그 나라를 오늘 하루 동안은 연합이라고 한다.
- 연합을 이루고 있는 각 칸의 인구수는 (연합의 인구수) / (연합을 이루고 있는 칸의 개수)가 된다. 편의상 소수점은 버린다.
- 연합을 해체하고, 모든 국경선을 닫는다.

인구수가 주어졌을 때, 인구 이동이 며칠 동안 발생하는지 구해라

- 인구이동이 없을 때까지 반복
- bfs로 방문하여 연합 진행

In [68]:
import math
from collections import deque

n, l, r = map(int, input().split())
graph = []
for _ in range(n):
    population = list(map(int, input().split()))
    graph.append(population)

# 이동할 네 방향 정의(상, 하, 좌, 우)
dx = [-1, 1, 0, 0]
dy = [0, 0, -1, 1]

def bfs(x, y):
    q = deque()
    q.append((x, y))
    visit[x][y] = True
    
    union = [(x, y)]
    cnt = graph[x][y]
    
    while q:
        x, y = q.popleft()
        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]
            if nx < 0 or ny < 0 or nx >= n or ny >= n:
                continue
            if visit[nx][ny]:
                continue
            # 해당 노드를 처음 방문하는 경우에만 최단 거리 기록
            if l <= abs(graph[nx][ny] - graph[x][y]) <= r: # 인구차이가 l명 이상 r명 이하인 경우:
                union.append((nx, ny))
                visit[nx][ny] = True
                q.append((nx, ny))
                cnt += graph[nx][ny]
                
    for x, y in union:
        graph[x][y] = math.trunc(cnt / len(union))
    
    return len(union)

days = 0
while True: # 인구 이동이 없을때까지 반복
    visit = [[False] * n for _ in range(n)]
    flag = False # 인구 이동 존재 유무
    for i in range(n):
        for j in range(n):
            if not visit[i][j]:
                if bfs(i, j) > 1:
                    flag = True
    if not flag: # 인구이동이 없는 경우
        break
    days += 1
    
print(days)

2 20 50
50 30
20 40
1


# 테트로미노

In [79]:
# 세로, 가로
n, m = map(int, input().split())

board = []
for _ in range(n):
    board.append(list(map(int, input().split())))

tetromino = [
    [(0,0), (0,1), (1,0), (1,1)], # ㅁ
    [(0,0), (0,1), (0,2), (0,3)], # ㅡ
    [(0,0), (1,0), (2,0), (3,0)], # ㅣ
    [(0,0), (0,1), (0,2), (1,0)], 
    [(1,0), (1,1), (1,2), (0,2)],
    [(0,0), (1,0), (1,1), (1,2)], # ㄴ
    [(0,0), (0,1), (0,2), (1,2)], # ㄱ
    [(0,0), (1,0), (2,0), (2,1)],
    [(2,0), (2,1), (1,1), (0,1)],
    [(0,0), (0,1), (1,0), (2,0)], 
    [(0,0), (0,1), (1,1), (2,1)],
    [(0,0), (0,1), (0,2), (1,1)], # ㅜ
    [(1,0), (1,1), (1,2), (0,1)], # ㅗ
    [(0,0), (1,0), (2,0), (1,1)], # ㅏ
    [(1,0), (0,1), (1,1), (2,1)], # ㅓ
    [(1,0), (2,0), (0,1), (1,1)],
    [(0,0), (1,0), (1,1), (2,1)],
    [(1,0), (0,1), (1,1), (0,2)],
    [(0,0), (0,1), (1,1), (1,2)]
]


def find(x, y):
    global answer
    for i in range(19): # 테트로미노 19가지 모양
        result = 0 # 각 테트로미노의 합산 값을 더합니다.
        for j in range(4): # 테트로미노는 4개의 블락으로 구성됩니다.
            try:
                next_x = x+tetromino[i][j][0] # 현재 위치에서 테트로미노를 놓은 x 좌표
                next_y = y+tetromino[i][j][1] # 현재 위치에서 테트로미노를 놓은 y 좌표
                result += board[next_x][next_y] # 합산 값을 구합니다.
            except IndexError: # 현재 위치에서 테트로미노가 board 밖으로 나가게 된다면 인덱스 에러 발생
                continue # 패스 
        answer = max(answer, result) # 최대 값을 저장

def solve():
    for i in range(n):
        for j in range(m):
            find(i, j)
            
answer = 0
solve()
print(answer)

5 5
1 2 3 4 5
5 4 3 2 1
2 3 4 5 6
6 5 4 3 2
1 2 1 2 1
19
